In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows",15)
%matplotlib inline

In [ ]:
class dataset:
    kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
    kdd_train_2labels_y = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
    
    kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")
    kdd_test_2labels_y = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")
    
    kdd_train_5labels = pd.read_pickle("dataset/kdd_train_5labels.pkl")
    kdd_train_5labels_y = pd.read_pickle("dataset/kdd_train_5labels_y.pkl")
    
    kdd_test_5labels = pd.read_pickle("dataset/kdd_test_5labels.pkl")
    kdd_test_5labels_y = pd.read_pickle("dataset/kdd_test_5labels_y.pkl")
    

In [ ]:
import collections
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import ShuffleSplit
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn import metrics
import time 

#names_noscaling = ["RBF SVM", "Decision Tree", "Random Forest", "Naive Bayes"]
names_noscaling = ["Naive Bayes", "Decision Tree", "Random Forest"]
classifiers_noscaling = [
                SVC(),
                DecisionTreeClassifier(max_depth=5),
                RandomForestClassifier(max_depth=5, n_estimators=10),
                GaussianNB()]

names_withscaling = ["Non - Linear SVM", "AdaBoost"]
classifiers_withscaling = [NuSVC(),
                AdaBoostClassifier()]

score = collections.namedtuple("score", ["name", "valid_score" ,"test_score", 'time_taken'])
scores = []
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)

for name, clf in zip(names_noscaling, classifiers_noscaling):
    print("Classifier: {}".format(name))
    start_time = time.perf_counter()
    
    x_train, y_train = dataset.kdd_train_2labels.iloc[:,:-2], dataset.kdd_train_2labels_y
    x_test, y_test = dataset.kdd_test_2labels.iloc[:,:-2], dataset.kdd_test_2labels_y
    
    clf_p = make_pipeline(clf)    
    valid_score = cross_val_score(clf_p, x_train, y_train, cv=cv)
    
    scaler = preprocessing.StandardScaler().fit(x_train)
    clf.fit(scaler.transform(x_train), y_train)
    y_pred = clf.predict(x_test)
    test_acc = metrics.accuracy_score(y_test, y_pred) 
    
    scores.append(score(name, valid_score.mean(), test_acc, time.perf_counter() - start_time))
    print("Score: {} \n".format(test_acc))

for name, clf in zip(names_withscaling, classifiers_withscaling):
    print("Classifier: {}".format(name))
    
    x_train, y_train = dataset.kdd_train_2labels.iloc[:,:-2], dataset.kdd_train_2labels_y
    x_test, y_test = dataset.kdd_test_2labels.iloc[:,:-2], dataset.kdd_test_2labels_y
    
    clf_p = make_pipeline(preprocessing.StandardScaler(), clf)    
    valid_score = cross_val_score(clf_p, x_train, y_train, cv=cv)
    
    scaler = preprocessing.StandardScaler().fit(x_train)
    clf.fit(scaler.transform(x_train), y_train)
    y_pred = clf.predict(x_test)
    test_acc = metrics.accuracy_score(y_test, y_pred) 
    
    scores.append(score(name, valid_score.mean(), test_acc))
    print("Score: {} \n".format(test_acc))


Classifier: Naive Bayes


In [ ]:
pd.DataFrame(scores)

In [ ]:
#Naive Bayes 3.4GB RAM